In [1]:
import pandas as pd
import os
from datetime import date, datetime,timedelta
from sqlalchemy import create_engine, text

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

format_dict = {'qty':'{:,}',
              'sell_price':'{:,.2f}','buy_price':'{:,.2f}','diff':'{:,.2f}',
              'sell_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}',
              'pct':'{:,.2f}','net':'{:,.2f}'}
pd.set_option('display.float_format','{:,.2f}'.format)
year = 2025
refund = 57_000   # 66,490.89 Windfall = 200,000 March
                    # LTF = 181498.8 + 3396.97 + 50380.22 = 235,000 June
                    # 501_000 + 167_000 = 668_000
dividend = 319_000 #361_000
expense  = 430_000

today = date.today()
today

datetime.date(2025, 9, 14)

### Restart and Run All Cells

In [3]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")
#C:\Users\PC1\OneDrive\A5\Excel
xsl_path = os.path.join(base_path, "Excel")

In [4]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path (dat_path): {dat_path}") 
print(f"Excel path (xsl_path): {xsl_path}") 
print(f"Google Drive path (god_path): {god_path}")
print(f"iCloudDrive path (icd_path): {icd_path}") 
print(f"Obsidian path (osd_path): {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\YTD Net Profit
Base path: C:\Users\PC1\OneDrive\A5
Data path (dat_path): C:\Users\PC1\OneDrive\A5\Data
Excel path (xsl_path): C:\Users\PC1\OneDrive\A5\Excel
Google Drive path (god_path): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path (icd_path): C:\Users\PC1\iCloudDrive\Data
Obsidian path (osd_path): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


### Record selection for transactions

In [6]:
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit AS net, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = %s
ORDER BY sells.date DESC, name'''
sql = sql % year
sells_df = pd.read_sql(sql, conpf)
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit AS net, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2025
ORDER BY sells.date DESC, name


In [7]:
sells_df.style.format(format_dict)

,name,sell_year,sell_month,buy_year,buy_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,net,kind
0,CPF,2025,9,2025,9,23.40,23.20,0.20,"5,000","117,000.00","116,000.00","1,000.00",0.86,483.93,DTD
1,CPF,2025,9,2025,8,23.40,24.10,-0.70,"5,000","117,000.00","120,500.00","-3,500.00",-2.90,"-4,026.04",DTD
2,DIF,2025,9,2025,4,8.30,7.50,0.80,"5,000","41,500.00","37,500.00","4,000.00",10.67,"3,825.02",HD
3,ORI,2025,8,2025,8,2.30,2.20,0.10,"5,000","11,500.00","11,000.00",500.00,4.55,450.16,DTD
4,MCS,2025,8,2025,2,8.40,7.30,1.10,"6,000","50,400.00","43,800.00","6,600.00",15.07,"6,391.35",HD
5,CPF,2025,8,2025,8,24.30,24.10,0.20,"5,000","121,500.00","120,500.00","1,000.00",0.83,463.98,DTD
6,CPF,2025,8,2025,8,24.30,24.10,0.20,"4,000","97,200.00","96,400.00",800.00,0.83,371.20,DTD
7,CPF,2025,8,2025,8,24.30,24.10,0.20,"4,000","97,200.00","96,400.00",800.00,0.83,371.20,DTD
8,GVREIT,2025,8,2023,9,6.70,6.10,0.60,"6,000","40,200.00","36,600.00","3,600.00",9.84,"3,429.91",HD
9,RCL,2025,8,2022,10,31.00,27.75,3.25,"3,000","93,000.00","83,250.00","9,750.00",11.71,"9,359.62",HD


In [8]:
sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum()

sell_year  sell_month  buy_year  buy_month  name  
2025       2           2021      8          TMT      -119,859.82
                                 10         TMT       -85,063.93
                       2022      6          TMT       -57,483.99
                       2023      2          ASK      -106,807.24
                       2024      10         ASK        -7,039.29
           3           2025      2          ORI          -926.69
           4           2022      2          KCE       -45,072.99
           5           2022      3          ASP       -55,283.40
           6           2022      2          KCE      -151,356.39
           7           2021      10         KCE      -129,185.74
                                 12         KCE      -130,582.63
                       2022      1          KCE      -121,848.74
                       2025      5          WHAIR       7,774.08
                                 6          IVL         5,048.96
                                       

In [9]:
sells_df.groupby(['name'])[['gross','net']].sum().head(5)

,gross,net
name,,
ASK,"-113,400.00","-113,846.53"
ASP,"-54,900.00","-55,283.40"
CPF,100.00,"-2,335.73"
DIF,"4,000.00","3,825.02"
GVREIT,"3,600.00","3,429.91"


In [10]:
#total_df = sells_df.groupby(['name'])[['gross','net']].sum()
#total_df.nlargest(5, 'gross')[['gross','net']].style.format(format_dict)

In [11]:
file_name = 'ytd-profit.csv'
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [12]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file : C:\Users\PC1\OneDrive\A5\Data\ytd-profit.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\ytd-profit.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\ytd-profit.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\ytd-profit.csv


In [13]:
sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum().to_csv(output_file)
sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum().to_csv(god_file)
sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum().to_csv(icd_file)
sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum().to_csv(osd_file)

In [14]:
output = sells_df.groupby(['sell_month']).net.sum()
output

sell_month
2   -376,254.27
3       -926.69
4    -45,072.99
5    -55,283.40
6   -151,356.39
7   -365,900.40
8     29,264.60
9        282.91
Name: net, dtype: float64

In [15]:
file_name = 'tmp-file-of-profits.csv'
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [16]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}")

Output file : C:\Users\PC1\OneDrive\A5\Data\tmp-file-of-profits.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\tmp-file-of-profits.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\tmp-file-of-profits.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\tmp-file-of-profits.csv


In [17]:
output.to_csv(output_file, index=True)
output.to_csv(god_file, index=True)
output.to_csv(icd_file, index=True)
output.to_csv(osd_file, index=True)

In [18]:
profit = sells_df.net.sum()
ttl_profit = profit + dividend
profit,dividend,ttl_profit

(-965246.6300000001, 319000, -646246.6300000001)

In [19]:
profit = sells_df.net.sum()
net = profit + dividend - expense
net = round(net / 1000,0) * 1000
round(profit,0),dividend,expense, round(net,0)

(-965247.0, 319000, 430000, -1076000.0)

### Bottom line of up to date transactions

In [21]:
trade = round(profit,0) + dividend
trade = round(trade / 1000,0) * 1000
trade

-646000.0

In [22]:
total = refund + net
refund, round(net,0), round(total,0)

(57000, -1076000.0, -1019000.0)

In [23]:
current_time = datetime.now()
formatted_time = current_time.strftime("%Y:%m:%d %H:%M:%S")
print(formatted_time)

2025:09:14 23:26:41
